# This ipynb file covers all the data collection and pre-processing for European Spotify datatset

# Importing all the required packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy import nan

import time
import random
from bs4 import BeautifulSoup as bs
import bs4 as bs
from bs4 import SoupStrainer

import requests
import random

# Our problem leverages data from two sources 

## - final_spotify.csv - This dataset has daily streaming count of different songs in Spotify along with weather details for the day
### Link for final_spotify.csv: https://www.kaggle.com/datasets/lautytonks/top-200-spotify-european-song-and-weather-data

## - songs.csv - This dataset has Sub-genre details of each song in Spotify
### Link for songs.csv: https://www.kaggle.com/datasets/nikitricky/every-noise-at-once

In [4]:
spotify_df = pd.read_csv("C:/Users/Thannir/Documents/WDA/final_spotify.csv")
genre_df = pd.read_csv("C:/Users/Thannir/Documents/WDA/songs.csv")

In [5]:
spotify_df.head()

,Unnamed: 0,region,date,month,spotify_id,artist,track_name,position,streams,danceability,...,acousticness,tempo,valence,explicit,temp,rain,snow,cloud,humidity,const
0,0,AUT,2019-01-02,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,35174,0.78,...,0.48,100.08,0.26,1.0,-5,1,1,48,69,1
1,1,AUT,2019-01-03,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,34237,0.78,...,0.48,100.08,0.26,1.0,-3,1,1,91,70,1
2,2,AUT,2019-01-04,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,35199,0.78,...,0.48,100.08,0.26,1.0,-2,0,0,14,75,1
3,3,AUT,2019-01-05,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,32559,0.78,...,0.48,100.08,0.26,1.0,0,0,0,47,80,1
4,4,AUT,2019-01-06,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,26956,0.78,...,0.48,100.08,0.26,1.0,0,1,0,80,74,1


In [6]:
genre_df.head()

,Name,Id,Artists,Artists_Id,Release,Duration,Genre,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valeance,Tempo
0,Don't Go Yet,1058fW9H3fZA6QjYCdOBad,['Camila Cabello'],['4nDoRrQiYLoBzwC5BhVJzF'],2021-07-23,164842,pop,0.666,0.796,10,-6.967,0.1030,0.0492,0.000000,0.0442,0.6100,110.108
1,What A Time (feat. Niall Horan),5WtfUKzXircvW8l5aaVZWT,"['Julia Michaels', 'Niall Horan']","['0ZED1XzwlLHW4ZaG4lOT6m', '1Hsdzj7Dlq2I7tHP75...",2019-01-24,173307,pop,0.516,0.467,0,-6.184,0.0302,0.6620,0.000000,0.0853,0.3860,132.861
2,Scars To Your Beautiful,42ydLwx4i5V49RXHOozJZq,['Alessia Cara'],['2wUjUUtkb5lvLKcGKsKqsR'],2016-03-11,230227,pop,0.319,0.739,0,-5.740,0.2720,0.0285,0.000000,0.1110,0.4490,194.169
3,Reckless,5ajjAnNRh8bxFvaVHzpPjh,['Madison Beer'],['2kRfqPViCqYdSGhYSM9R0Q'],2021-06-04,203385,pop,0.386,0.426,3,-6.642,0.0363,0.8070,0.000000,0.1400,0.2610,180.104
4,Dusk Till Dawn (feat. Sia) - Radio Edit,3e7sxremeOE3wTySiOhGiP,"['ZAYN', 'Sia']","['5ZsFI1h6hIdQRw2ti0hz81', '5WUlDfRSoLAfcVSX1W...",2018-12-14,239000,pop,0.259,0.437,11,-6.589,0.0386,0.1020,0.000001,0.1060,0.0951,180.042


# Performing a left join to get the Sub-genre details for each song in spotify_df

In [7]:
df_left = spotify_df.merge(genre_df[["Genre", "Id"]],  how = 'left', left_on = ['spotify_id'], right_on = ['Id'])
df_left.head()

,Unnamed: 0,region,date,month,spotify_id,artist,track_name,position,streams,danceability,...,valence,explicit,temp,rain,snow,cloud,humidity,const,Genre,Id
0,0,AUT,2019-01-02,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,35174,0.78,...,0.26,1.0,-5,1,1,48,69,1,NaN,NaN
1,1,AUT,2019-01-03,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,34237,0.78,...,0.26,1.0,-3,1,1,91,70,1,NaN,NaN
2,2,AUT,2019-01-04,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,35199,0.78,...,0.26,1.0,-2,0,0,14,75,1,NaN,NaN
3,3,AUT,2019-01-05,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,32559,0.78,...,0.26,1.0,0,0,0,47,80,1,NaN,NaN
4,4,AUT,2019-01-06,1,7KPGeiXWDsGEvFK62wzU8E,Capital Bra,Benzema,1,26956,0.78,...,0.26,1.0,0,1,0,80,74,1,NaN,NaN


## Removing rows which didn't have a sub-genre match in left join

In [8]:
df_left_nonNAN = df_left.dropna(subset=["Genre"])
df_left_nonNAN.head()

,Unnamed: 0,region,date,month,spotify_id,artist,track_name,position,streams,danceability,...,valence,explicit,temp,rain,snow,cloud,humidity,const,Genre,Id
27674,27993,AUT,2019-01-02,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13742,0.69,...,0.16,0.0,-5,1,1,48,69,1,pop dance,0E9ZjEAyAwOXZ7wJC0PD33
27675,27994,AUT,2019-01-03,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13538,0.69,...,0.16,0.0,-3,1,1,91,70,1,pop dance,0E9ZjEAyAwOXZ7wJC0PD33
27676,27995,AUT,2019-01-04,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,15,13870,0.69,...,0.16,0.0,-2,0,0,14,75,1,pop dance,0E9ZjEAyAwOXZ7wJC0PD33
27677,27996,AUT,2019-01-05,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13233,0.69,...,0.16,0.0,0,0,0,47,80,1,pop dance,0E9ZjEAyAwOXZ7wJC0PD33
27678,27997,AUT,2019-01-06,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,16,10903,0.69,...,0.16,0.0,0,1,0,80,74,1,pop dance,0E9ZjEAyAwOXZ7wJC0PD33


## Distinct count of sub-genre & dimensions of the cleaned dataset

In [9]:
print("Unique count of sub-genre: ",df_left_nonNAN['Genre'].nunique())
print("(Rows, Columns): ", df_left_nonNAN.shape)

Unique count of sub-genre:  537
(Rows, Columns):  (284264, 28)


In [10]:
df_genre_counts = df_left_nonNAN['Genre'].value_counts()
df_genre_counts

#df_genre_counts.to_csv('df_genre_counts.csv')

pop                  25584
uk pop               14772
pop dance            11202
melodic rap          10731
rap                   9108
                     ...  
indie viet               1
traprun                  1
mississippi indie        1
taiwanese indie          1
hip hop boliviano        1
Name: Genre, Length: 537, dtype: int64

# Since there are 537 sub-genre, analysing them to see if daily weather has an impact on the type (genre) of songs played might not yeild the right or significant results.
# So, we will scrape data from "https://www.chosic.com/list-of-music-genres/" which has data to map each sub-genre to a parent genre. It will map all the sub-genre to a total list of 15 parent genre.

In [11]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

headers = random.choice(headers_list)
r = requests.Session()
r.headers = headers

In [12]:
genres=[]
sub_genres=[]
num=0

link= 'https://www.chosic.com/list-of-music-genres/'
html = r.get(link).text

html=html[html.find('Sub genres'):]

while html.find('yoik') != -1:
    soup=bs.BeautifulSoup(html,'html.parser')
    genre= soup.find('li',class_='genre-term-basic')
    print('***', genre.text ,'***','\n')
    
    
    a= soup.find('ul',class_='ul-inside')
    for i in a.find_all('li',class_='capital-letter genre-term'):
        subgenre= i.find('a')
        sub_genres.append(subgenre.text)
        genres.append(genre.text)
        print(subgenre.text)
    print('_'*50)
    
    num= num+1
    x= html.find('genre'+str(num))
    html= html[x-20:]

*** Pop *** 

pop
acoustic pop
arab pop
art pop
c-pop
classify
dance pop
electropop
escape room
europop
hip pop
hyperpop
indie cafe pop
indie electropop
indie pop
indie poptimism
j-pop
k-pop
latin pop
levenslied
metropopolis
mexican pop
neo mellow
new romantic
new wave pop
pop dance
pop edm
pop r&b
pop rap
pop rock
post-teen pop
social media pop
soft rock
sophisti-pop
talent show
teen pop
turkish pop
vapor soul
viral pop
vispop
__________________________________________________
*** Dance / EDM (Electronic Dance Music) *** 

edm
bass house
bass trap
big room
breakbeat
breakcore
brostep
chillstep
complextro
deep big room
deep groove house
deep house
deep tropical house
disco house
dubstep
electro house
electronic trap
electropop
electro swing
filthstep
future bass
future garage
future house
gaming dubstep
gaming edm
glitch hop
house
indie electropop
melodic dubstep
pop dance
pop edm
progressive electro house
progressive house
progressive trance
sky room
tech house
trance
tropical house
u

### Genre list

In [13]:
genres

['Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Pop',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electronic Dance Music)',
 'Dance / EDM (Electroni

### Sub-genre list

In [14]:
sub_genres

['pop',
 'acoustic pop',
 'arab pop',
 'art pop',
 'c-pop',
 'classify',
 'dance pop',
 'electropop',
 'escape room',
 'europop',
 'hip pop',
 'hyperpop',
 'indie cafe pop',
 'indie electropop',
 'indie pop',
 'indie poptimism',
 'j-pop',
 'k-pop',
 'latin pop',
 'levenslied',
 'metropopolis',
 'mexican pop',
 'neo mellow',
 'new romantic',
 'new wave pop',
 'pop dance',
 'pop edm',
 'pop r&b',
 'pop rap',
 'pop rock',
 'post-teen pop',
 'social media pop',
 'soft rock',
 'sophisti-pop',
 'talent show',
 'teen pop',
 'turkish pop',
 'vapor soul',
 'viral pop',
 'vispop',
 'edm',
 'bass house',
 'bass trap',
 'big room',
 'breakbeat',
 'breakcore',
 'brostep',
 'chillstep',
 'complextro',
 'deep big room',
 'deep groove house',
 'deep house',
 'deep tropical house',
 'disco house',
 'dubstep',
 'electro house',
 'electronic trap',
 'electropop',
 'electro swing',
 'filthstep',
 'future bass',
 'future garage',
 'future house',
 'gaming dubstep',
 'gaming edm',
 'glitch hop',
 'house',
 

## Creating a separate list for each parent genre. Each parent genre list will contain the sub-genre present under them.

In [15]:
Pop = []
Dance_EDM = []
Hiphop_and_Rap = []
RandB = []
Latin = []
Rock = []
Metal = []
Country = []
Folk_Acoustic = []
Classical = []
Jazz = []
Blues = []
Easy_Listening = []
New_Age = []
World_Traditional_Folk = []

for i in range(len(sub_genres)):
    if genres[i] == "Pop":
        Pop.append(sub_genres[i])
        
    elif genres[i] == "Dance / EDM (Electronic Dance Music)":
        Dance_EDM.append(sub_genres[i])
        
    elif genres[i] == "Hip-hop and Rap":
        Hiphop_and_Rap.append(sub_genres[i])
        
    elif genres[i] == "R&B":
        RandB.append(sub_genres[i])
        
    elif genres[i] == "Latin":
        Latin.append(sub_genres[i])
        
    elif genres[i] == "Rock":
        Rock.append(sub_genres[i])
        
    elif genres[i] == "Metal":
        Metal.append(sub_genres[i])
        
    elif genres[i] == "Country":
        Country.append(sub_genres[i])
        
    elif genres[i] == "Folk / Acoustic (also called Contemporary folk - wikipedia)":
        Folk_Acoustic.append(sub_genres[i])
        
    elif genres[i] == "Classical":
        Classical.append(sub_genres[i])
        
    elif genres[i] == "Jazz":
        Jazz.append(sub_genres[i])
        
    elif genres[i] == "Blues":
        Blues.append(sub_genres[i])
        
    elif genres[i] == "Easy Listening":
        Easy_Listening.append(sub_genres[i])
        
    elif genres[i] == "New Age":
        New_Age.append(sub_genres[i])
        
    elif genres[i] == "World / Traditional Folk (wikipedia)":
        World_Traditional_Folk.append(sub_genres[i])

print(Pop)
print(Dance_EDM)
print(Hiphop_and_Rap)
print(RandB)
print(Latin)
print(Rock)
print(Metal)
print(Country)
print(Folk_Acoustic)
print(Classical)
print(Jazz)
print(Blues)
print(Easy_Listening)
print(New_Age)
print(World_Traditional_Folk)

['pop', 'acoustic pop', 'arab pop', 'art pop', 'c-pop', 'classify', 'dance pop', 'electropop', 'escape room', 'europop', 'hip pop', 'hyperpop', 'indie cafe pop', 'indie electropop', 'indie pop', 'indie poptimism', 'j-pop', 'k-pop', 'latin pop', 'levenslied', 'metropopolis', 'mexican pop', 'neo mellow', 'new romantic', 'new wave pop', 'pop dance', 'pop edm', 'pop r&b', 'pop rap', 'pop rock', 'post-teen pop', 'social media pop', 'soft rock', 'sophisti-pop', 'talent show', 'teen pop', 'turkish pop', 'vapor soul', 'viral pop', 'vispop']
['edm', 'bass house', 'bass trap', 'big room', 'breakbeat', 'breakcore', 'brostep', 'chillstep', 'complextro', 'deep big room', 'deep groove house', 'deep house', 'deep tropical house', 'disco house', 'dubstep', 'electro house', 'electronic trap', 'electropop', 'electro swing', 'filthstep', 'future bass', 'future garage', 'future house', 'gaming dubstep', 'gaming edm', 'glitch hop', 'house', 'indie electropop', 'melodic dubstep', 'pop dance', 'pop edm', 'pr

## Creating a list of sub-genre present in the cleaned dataset (spotify_df)

In [16]:
sub_genre = list(df_left_nonNAN.Genre)
sub_genre = [i.lower() for i in sub_genre]
sub_genre

['pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop dance',
 'pop 

## One-hot encoding to each parent genre (creating one column each for all parent genre with 1,0 indicating if that record is for that parent genre)

In [17]:
Pop_Flag = []
Dance_EDM_Flag = []
Hiphop_and_Rap_Flag = []
RandB_Flag = []
Latin_Flag = []
Rock_Flag = []
Metal_Flag = []
Country_Flag = []
Folk_Acoustic_Flag = []
Classical_Flag = []
Jazz_Flag = []
Blues_Flag = []
Easy_Listening_Flag = []
New_Age_Flag = []
World_Traditional_Folk_Flag = []

for i in sub_genre:
    if i in Pop:
        Pop_Flag.append(1)
    else:
        Pop_Flag.append(0)

for i in sub_genre:
    if i in Dance_EDM:
        Dance_EDM_Flag.append(1)
    else:
        Dance_EDM_Flag.append(0)

for i in sub_genre:
    if i in Hiphop_and_Rap:
        Hiphop_and_Rap_Flag.append(1)
    else:
        Hiphop_and_Rap_Flag.append(0)

for i in sub_genre:
    if i in RandB:
        RandB_Flag.append(1)
    else:
        RandB_Flag.append(0)

for i in sub_genre:
    if i in Latin:
        Latin_Flag.append(1)
    else:
        Latin_Flag.append(0)

for i in sub_genre:
    if i in Rock:
        Rock_Flag.append(1)
    else:
        Rock_Flag.append(0)

for i in sub_genre:
    if i in Metal:
        Metal_Flag.append(1)
    else:
        Metal_Flag.append(0)

for i in sub_genre:
    if i in Country:
        Country_Flag.append(1)
    else:
        Country_Flag.append(0)

for i in sub_genre:
    if i in Folk_Acoustic:
        Folk_Acoustic_Flag.append(1)
    else:
        Folk_Acoustic_Flag.append(0)
        
for i in sub_genre:
    if i in Classical:
        Classical_Flag.append(1)
    else:
        Classical_Flag.append(0)
        
for i in sub_genre:
    if i in Jazz:
        Jazz_Flag.append(1)
    else:
        Jazz_Flag.append(0)
        
for i in sub_genre:
    if i in Blues:
        Blues_Flag.append(1)
    else:
        Blues_Flag.append(0)
        
for i in sub_genre:
    if i in Easy_Listening:
        Easy_Listening_Flag.append(1)
    else:
        Easy_Listening_Flag.append(0)
        
for i in sub_genre:
    if i in New_Age:
        New_Age_Flag.append(1)
    else:
        New_Age_Flag.append(0)
        
for i in sub_genre:
    if i in World_Traditional_Folk:
        World_Traditional_Folk_Flag.append(1)
    else:
        World_Traditional_Folk_Flag.append(0)

In [18]:
df_left_nonNAN['Pop_Flag'] = Pop_Flag
df_left_nonNAN['Dance_EDM_Flag'] = Dance_EDM_Flag
df_left_nonNAN['Hiphop_and_Rap_Flag'] = Hiphop_and_Rap_Flag
df_left_nonNAN['RandB_Flag'] = RandB_Flag
df_left_nonNAN['Latin_Flag'] = Latin_Flag
df_left_nonNAN['Rock_Flag'] = Rock_Flag
df_left_nonNAN['Metal_Flag'] = Metal_Flag
df_left_nonNAN['Country_Flag'] = Country_Flag
df_left_nonNAN['Folk_Acoustic_Flag'] = Folk_Acoustic_Flag
df_left_nonNAN['Classical_Flag'] = Classical_Flag
df_left_nonNAN['Jazz_Flag'] = Jazz_Flag
df_left_nonNAN['Blues_Flag'] = Blues_Flag
df_left_nonNAN['Easy_Listening_Flag'] = Easy_Listening_Flag
df_left_nonNAN['New_Age_Flag'] = New_Age_Flag
df_left_nonNAN['World_Traditional_Folk_Flag'] = World_Traditional_Folk_Flag

df_left_nonNAN

C:\Users\Thannir\AppData\Local\Temp/ipykernel_9924/1529066289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_nonNAN['Pop_Flag'] = Pop_Flag
C:\Users\Thannir\AppData\Local\Temp/ipykernel_9924/1529066289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_nonNAN['Dance_EDM_Flag'] = Dance_EDM_Flag
C:\Users\Thannir\AppData\Local\Temp/ipykernel_9924/1529066289.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Unnamed: 0,region,date,month,spotify_id,artist,track_name,position,streams,danceability,...,Rock_Flag,Metal_Flag,Country_Flag,Folk_Acoustic_Flag,Classical_Flag,Jazz_Flag,Blues_Flag,Easy_Listening_Flag,New_Age_Flag,World_Traditional_Folk_Flag
27674,27993,AUT,2019-01-02,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13742,0.69,...,0,0,0,0,0,0,0,0,0,0
27675,27994,AUT,2019-01-03,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13538,0.69,...,0,0,0,0,0,0,0,0,0,0
27676,27995,AUT,2019-01-04,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,15,13870,0.69,...,0,0,0,0,0,0,0,0,0,0
27677,27996,AUT,2019-01-05,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13233,0.69,...,0,0,0,0,0,0,0,0,0,0
27678,27997,AUT,2019-01-06,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,16,10903,0.69,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545614,1437623,SVK,2019-12-27,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,119,1587,0.71,...,0,0,0,0,0,0,0,0,0,0
1545615,1437624,SVK,2019-12-28,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,136,1410,0.71,...,0,0,0,0,0,0,0,0,0,0
1545616,1437625,SVK,2019-12-29,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,110,1544,0.71,...,0,0,0,0,0,0,0,0,0,0
1545617,1437626,SVK,2019-12-30,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,149,1489,0.71,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#df_left_nonNAN.to_csv('df_left_nonNAN_updated.csv')

## Calculated column of flag column * number of streams for each parent genre

In [20]:
df_parent_genre_streams = df_left_nonNAN

df_parent_genre_streams['Pop_Streams'] = df_parent_genre_streams['Pop_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Dance_EDM_Streams'] = df_parent_genre_streams['Dance_EDM_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Hiphop_and_Rap_Streams'] = df_parent_genre_streams['Hiphop_and_Rap_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['RandB_Streams'] = df_parent_genre_streams['RandB_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Latin_Streams'] = df_parent_genre_streams['Latin_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Rock_Streams'] = df_parent_genre_streams['Rock_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Metal_Streams'] = df_parent_genre_streams['Metal_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Country_Streams'] = df_parent_genre_streams['Country_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Folk_Acoustic_Streams'] = df_parent_genre_streams['Folk_Acoustic_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Classical_Streams'] = df_parent_genre_streams['Classical_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Jazz_Streams'] = df_parent_genre_streams['Jazz_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Blues_Streams'] = df_parent_genre_streams['Blues_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['Easy_Listening_Streams'] = df_parent_genre_streams['Easy_Listening_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['New_Age_Streams'] = df_parent_genre_streams['New_Age_Flag'] * df_parent_genre_streams['streams']
df_parent_genre_streams['World_Traditional_Folk_Streams'] = df_parent_genre_streams['World_Traditional_Folk_Flag'] * df_parent_genre_streams['streams']

df_parent_genre_streams

C:\Users\Thannir\AppData\Local\Temp/ipykernel_9924/1117951968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parent_genre_streams['Pop_Streams'] = df_parent_genre_streams['Pop_Flag'] * df_parent_genre_streams['streams']
C:\Users\Thannir\AppData\Local\Temp/ipykernel_9924/1117951968.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parent_genre_streams['Dance_EDM_Streams'] = df_parent_genre_streams['Dance_EDM_Flag'] * df_parent_genre_streams['streams']
C:\Users\Thannir\AppData\Local\Temp/ipykerne

,Unnamed: 0,region,date,month,spotify_id,artist,track_name,position,streams,danceability,...,Rock_Streams,Metal_Streams,Country_Streams,Folk_Acoustic_Streams,Classical_Streams,Jazz_Streams,Blues_Streams,Easy_Listening_Streams,New_Age_Streams,World_Traditional_Folk_Streams
27674,27993,AUT,2019-01-02,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13742,0.69,...,0,0,0,0,0,0,0,0,0,0
27675,27994,AUT,2019-01-03,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13538,0.69,...,0,0,0,0,0,0,0,0,0,0
27676,27995,AUT,2019-01-04,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,15,13870,0.69,...,0,0,0,0,0,0,0,0,0,0
27677,27996,AUT,2019-01-05,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,14,13233,0.69,...,0,0,0,0,0,0,0,0,0,0
27678,27997,AUT,2019-01-06,1,0E9ZjEAyAwOXZ7wJC0PD33,Dynoro,In My Mind,16,10903,0.69,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545614,1437623,SVK,2019-12-27,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,119,1587,0.71,...,0,0,0,0,0,0,0,0,0,0
1545615,1437624,SVK,2019-12-28,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,136,1410,0.71,...,0,0,0,0,0,0,0,0,0,0
1545616,1437625,SVK,2019-12-29,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,110,1544,0.71,...,0,0,0,0,0,0,0,0,0,0
1545617,1437626,SVK,2019-12-30,12,1EJqKPvXJsNbXjURpTRU6D,Rida Radar,(M)Ona,149,1489,0.71,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#df_parent_genre_streams.to_csv('df_parent_genre_streams_updated.csv')

In [22]:
#df_parent_genre_streams[['Pop_Streams', 'Dance_EDM_Streams', 'Hiphop_and_Rap_Streams', 'RandB_Streams', 'Latin_Streams', 'Rock_Streams', 'Metal_Streams', 'Country_Streams', 'Folk_Acoustic_Streams', 'Classical_Streams', 'Jazz_Streams', 'Blues_Streams', 'Easy_Listening_Streams', 'New_Age_Streams', 'World_Traditional_Folk_Streams', 'temp', 'rain', 'snow', 'cloud', 'humidity']].corr()

In [23]:
df_parent_genre_streams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284264 entries, 27674 to 1545638
Data columns (total 58 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      284264 non-null  int64  
 1   region                          284264 non-null  object 
 2   date                            284264 non-null  object 
 3   month                           284264 non-null  int64  
 4   spotify_id                      284264 non-null  object 
 5   artist                          284263 non-null  object 
 6   track_name                      284264 non-null  object 
 7   position                        284264 non-null  int64  
 8   streams                         284264 non-null  int64  
 9   danceability                    284264 non-null  float64
 10  energy                          284264 non-null  float64
 11  instrumentalness                284264 non-null  float64
 12  key        

## Dropping flag and other additional columns

In [24]:
df_updated = df_parent_genre_streams.drop(['track_name', 'position', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'acousticness', 'tempo', 'valence', 'explicit', 'const', 'Id', 'Pop_Flag', 'Dance_EDM_Flag', 'Hiphop_and_Rap_Flag', 'RandB_Flag', 'Latin_Flag', 'Rock_Flag', 'Metal_Flag', 'Country_Flag', 'Folk_Acoustic_Flag', 'Classical_Flag', 'Jazz_Flag', 'Blues_Flag', 'Easy_Listening_Flag', 'New_Age_Flag', 'World_Traditional_Folk_Flag'],axis=1)
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284264 entries, 27674 to 1545638
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Unnamed: 0                      284264 non-null  int64 
 1   region                          284264 non-null  object
 2   date                            284264 non-null  object
 3   month                           284264 non-null  int64 
 4   spotify_id                      284264 non-null  object
 5   artist                          284263 non-null  object
 6   streams                         284264 non-null  int64 
 7   temp                            284264 non-null  int64 
 8   rain                            284264 non-null  int64 
 9   snow                            284264 non-null  int64 
 10  cloud                           284264 non-null  int64 
 11  humidity                        284264 non-null  int64 
 12  Genre                    

# Creating flags to identify different daily weather conditions: Sunny, Rainy, Cloudy, Snowy, Humid
### Reference: 
"https://www.mentalfloss.com/article/56820/whats-difference-between-mostly-sunny-and-partly-cloudy", "https://weather.com/science/weather-explainers/news/common-weather-terms-used-incorrectly", "https://www.weather.gov/arx/why_dewpoint_vs_humidity#:~:text=less%20than%20or%20equal%20to,in%20the%20air%2C%20becoming%20oppressive"

In [25]:
#less than or equal to 3/8th is SUnny
df_updated['Sunny'] = np.where(df_updated['cloud'] <= 37.5, '1', '0')

df_updated['Rainy'] = df_updated['rain'].apply(str)

#greater than or equal to 5/8th is Cloudy
df_updated['Cloudy'] = np.where(df_updated['cloud'] >= 62.5, '1', '0')

df_updated['Snowy'] = df_updated['snow'].apply(str)

#greater than 0.55 is Humid
df_updated['Humid'] = np.where(df_updated['humidity'] > 55, '1', '0')

df_updated = df_updated.drop(['temp', 'cloud', 'humidity', 'rain', 'snow'],axis=1)
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284264 entries, 27674 to 1545638
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Unnamed: 0                      284264 non-null  int64 
 1   region                          284264 non-null  object
 2   date                            284264 non-null  object
 3   month                           284264 non-null  int64 
 4   spotify_id                      284264 non-null  object
 5   artist                          284263 non-null  object
 6   streams                         284264 non-null  int64 
 7   Genre                           284264 non-null  object
 8   Pop_Streams                     284264 non-null  int64 
 9   Dance_EDM_Streams               284264 non-null  int64 
 10  Hiphop_and_Rap_Streams          284264 non-null  int64 
 11  RandB_Streams                   284264 non-null  int64 
 12  Latin_Streams            

## Aggregating the individual genre streams at a Region-Date level

In [26]:
df_updated_grouped = df_updated.groupby(['region', 'date']).agg(
             Pop_Streams_Sum = ('Pop_Streams', 'sum'),
             Dance_EDM_Streams_Sum = ('Dance_EDM_Streams', 'sum'),
             Hiphop_and_Rap_Streams_Sum = ('Hiphop_and_Rap_Streams', 'sum'),
             RandB_Streams_Sum = ('RandB_Streams', 'sum'),
             Latin_Streams_Sum = ('Latin_Streams', 'sum'),
             Rock_Streams_Sum = ('Rock_Streams', 'sum'),
             Metal_Streams_Sum = ('Metal_Streams', 'sum'),
             Country_Streams_Sum = ('Country_Streams', 'sum'),
             Folk_Acoustic_Streams_Sum = ('Folk_Acoustic_Streams', 'sum'),
             Classical_Streams_Sum = ('Classical_Streams', 'sum'),
             Jazz_Streams_Sum = ('Jazz_Streams', 'sum'),
             Blues_Streams_Sum = ('Blues_Streams', 'sum'),
             Easy_Listening_Streams_Sum = ('Easy_Listening_Streams', 'sum'),
             New_Age_Streams_Sum = ('New_Age_Streams', 'sum'),
             World_Traditional_Folk_Streams_Sum = ('World_Traditional_Folk_Streams', 'sum'),
             Sunny_Flag = ('Sunny', 'max'),
             Rainy_Flag = ('Rainy', 'max'),
             Cloudy_Flag = ('Cloudy', 'max'),
             Snowy_Flag = ('Snowy', 'max'),
             Humid_Flag = ('Humid', 'max'))

df_updated_grouped

Pop_Streams_Sum  Dance_EDM_Streams_Sum  \
region date                                                 
AUT    2019-01-02            61182                  35702   
       2019-01-03            60216                  35239   
       2019-01-04            61845                  36085   
       2019-01-05            58174                  35188   
       2019-01-06            51145                  29152   
...                            ...                    ...   
SWE    2019-12-27           133360                  35663   
       2019-12-28           129569                  34514   
       2019-12-29           123262                  33100   
       2019-12-30           143015                  40055   
       2019-12-31           652611                 232389   

                   Hiphop_and_Rap_Streams_Sum  RandB_Streams_Sum  \
region date                                                        
AUT    2019-01-02                       24773                  0   
       2019-01-03                       25646                  0   
       2019-01-04                       25997                  0   
       2019-01-05                       24777                  0   
       2019-01-06                       21185                  0   
...                                       ...                ...   
SWE    2019-12-27                       59410                  0   
       2019-12-28                       58080                  0   
       2019-12-29                       56229                  0   
       2019-12-30                       63462                  0   
       2019-12-31                      117732              34453   

                   Latin_Streams_Sum  Rock_Streams_Sum  Metal_Streams_Sum  \
region date                                                                 
AUT    2019-01-02              10982              3329                  0   
       2019-01-03              11390              6477                  0   
       2019-01-04              11835              3588                  0   
       2019-01-05              11233              7019                  0   
       2019-01-06               9537              5700                  0   
...                              ...               ...                ...   
SWE    2019-12-27              74399             19286                  0   
       2019-12-28              73792             19451                  0   
       2019-12-29              71821             17549                  0   
       2019-12-30              81256             21111                  0   
       2019-12-31             179175            153686              61853   

                   Country_Streams_Sum  Folk_Acoustic_Streams_Sum  \
region date                                                         
AUT    2019-01-02                    0                          0   
       2019-01-03                    0                          0   
       2019-01-04                    0                          0   
       2019-01-05                    0                          0   
       2019-01-06                    0                          0   
...                                ...                        ...   
SWE    2019-12-27                    0                      35663   
       2019-12-28                    0                      34514   
       2019-12-29                    0                      33100   
       2019-12-30                    0                      40055   
       2019-12-31                    0                      73845   

                   Classical_Streams_Sum  Jazz_Streams_Sum  Blues_Streams_Sum  \
region date                                                                     
AUT    2019-01-02                      0                 0                  0   
       2019-01-03                      0                 0                  0   
       2019-01-04                      0                 0                  0   
       2019-01-05                  

In [27]:
#df_updated_grouped.to_csv('df_updated_grouped.csv')

In [28]:
Total_Streams_Sum = df_updated_grouped['Pop_Streams_Sum'] + df_updated_grouped['Dance_EDM_Streams_Sum'] + df_updated_grouped['Hiphop_and_Rap_Streams_Sum'] + df_updated_grouped['RandB_Streams_Sum'] + df_updated_grouped['Latin_Streams_Sum'] + df_updated_grouped['Rock_Streams_Sum'] + df_updated_grouped['Metal_Streams_Sum'] + df_updated_grouped['Country_Streams_Sum'] + df_updated_grouped['Folk_Acoustic_Streams_Sum'] + df_updated_grouped['Classical_Streams_Sum'] + df_updated_grouped['Jazz_Streams_Sum'] + df_updated_grouped['Blues_Streams_Sum'] + df_updated_grouped['Easy_Listening_Streams_Sum'] + df_updated_grouped['New_Age_Streams_Sum'] + df_updated_grouped['World_Traditional_Folk_Streams_Sum']
sum(Total_Streams_Sum)

3752584440

# Identifying the major genres played in Spotify to analyze the impact of different daily weather conditions on them

In [29]:
print('Share of Pop_Streams: ', round((df_updated_grouped['Pop_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Latin_Streams: ', round((df_updated_grouped['Latin_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Hiphop_and_Rap_Streams: ', round((df_updated_grouped['Hiphop_and_Rap_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Dance_EDM_Streams: ', round((df_updated_grouped['Dance_EDM_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Rock_Streams: ', round((df_updated_grouped['Rock_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Folk_Acoustic_Streams: ', round((df_updated_grouped['Folk_Acoustic_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of RandB_Streams: ', round((df_updated_grouped['RandB_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Metal_Streams: ', round((df_updated_grouped['Metal_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Country_Streams: ', round((df_updated_grouped['Country_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Classical_Streams: ', round((df_updated_grouped['Classical_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Jazz_Streams: ', round((df_updated_grouped['Jazz_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Blues_Streams: ', round((df_updated_grouped['Blues_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of Easy_Listening_Streams: ', round((df_updated_grouped['Easy_Listening_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of New_Age_Streams: ', round((df_updated_grouped['New_Age_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')
print('Share of World_Traditional_Folk_Streams: ', round((df_updated_grouped['World_Traditional_Folk_Streams_Sum'].sum()/sum(Total_Streams_Sum))*100,2), '%')

print(' ')
print(' ')

print('Reason for going ahead with less genre:')
print('Top 6 parent genre make up for ', round(((df_updated_grouped['Pop_Streams_Sum'] + df_updated_grouped['Latin_Streams_Sum'] + df_updated_grouped['Hiphop_and_Rap_Streams_Sum'] + df_updated_grouped['Dance_EDM_Streams_Sum'] + df_updated_grouped['Rock_Streams_Sum'] + df_updated_grouped['Folk_Acoustic_Streams_Sum']).sum()/sum(Total_Streams_Sum))*100,0), '% of streams in Spotify Europe')

print(' ')
print(' ')

print('The top 6 genre are:\n 1. Pop\n 2. Latin\n 3. Hiphop_and_Rap\n 4. Dance_EDM\n 5. Rock\n 6. Folk_Acoustic')

Share of Pop_Streams:  28.94 %
Share of Latin_Streams:  26.86 %
Share of Hiphop_and_Rap_Streams:  14.88 %
Share of Dance_EDM_Streams:  11.92 %
Share of Rock_Streams:  7.24 %
Share of Folk_Acoustic_Streams:  5.08 %
Share of RandB_Streams:  1.35 %
Share of Metal_Streams:  0.12 %
Share of Country_Streams:  1.14 %
Share of Classical_Streams:  0.0 %
Share of Jazz_Streams:  0.64 %
Share of Blues_Streams:  0.55 %
Share of Easy_Listening_Streams:  1.1 %
Share of New_Age_Streams:  0.17 %
Share of World_Traditional_Folk_Streams:  0.01 %
 
 
Reason for going ahead with less genre:
Top 6 parent genre make up for  95.0 % of streams in Spotify Europe
 
 
The top 6 genre are:
 1. Pop
 2. Latin
 3. Hiphop_and_Rap
 4. Dance_EDM
 5. Rock
 6. Folk_Acoustic


In [30]:
df_final = df_updated_grouped

df_final['Total_Streams_Sum'] = df_final['Pop_Streams_Sum'] + df_final['Dance_EDM_Streams_Sum'] + df_final['Hiphop_and_Rap_Streams_Sum'] + df_final['Latin_Streams_Sum'] + df_final['Rock_Streams_Sum'] + df_final['Folk_Acoustic_Streams_Sum']

df_final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7277 entries, ('AUT', '2019-01-02') to ('SWE', '2019-12-31')
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Pop_Streams_Sum                     7277 non-null   int64 
 1   Dance_EDM_Streams_Sum               7277 non-null   int64 
 2   Hiphop_and_Rap_Streams_Sum          7277 non-null   int64 
 3   RandB_Streams_Sum                   7277 non-null   int64 
 4   Latin_Streams_Sum                   7277 non-null   int64 
 5   Rock_Streams_Sum                    7277 non-null   int64 
 6   Metal_Streams_Sum                   7277 non-null   int64 
 7   Country_Streams_Sum                 7277 non-null   int64 
 8   Folk_Acoustic_Streams_Sum           7277 non-null   int64 
 9   Classical_Streams_Sum               7277 non-null   int64 
 10  Jazz_Streams_Sum                    7277 non-null   int64 
 11  Blues_Streams_Sum  

In [31]:
df_final = df_final.drop(['RandB_Streams_Sum', 'Metal_Streams_Sum', 'Country_Streams_Sum', 'Classical_Streams_Sum', 'Jazz_Streams_Sum', 'Blues_Streams_Sum', 'Easy_Listening_Streams_Sum', 'New_Age_Streams_Sum', 'World_Traditional_Folk_Streams_Sum', 'Total_Streams_Sum'],axis=1)
df_final

Pop_Streams_Sum  Dance_EDM_Streams_Sum  \
region date                                                 
AUT    2019-01-02            61182                  35702   
       2019-01-03            60216                  35239   
       2019-01-04            61845                  36085   
       2019-01-05            58174                  35188   
       2019-01-06            51145                  29152   
...                            ...                    ...   
SWE    2019-12-27           133360                  35663   
       2019-12-28           129569                  34514   
       2019-12-29           123262                  33100   
       2019-12-30           143015                  40055   
       2019-12-31           652611                 232389   

                   Hiphop_and_Rap_Streams_Sum  Latin_Streams_Sum  \
region date                                                        
AUT    2019-01-02                       24773              10982   
       2019-01-03                       25646              11390   
       2019-01-04                       25997              11835   
       2019-01-05                       24777              11233   
       2019-01-06                       21185               9537   
...                                       ...                ...   
SWE    2019-12-27                       59410              74399   
       2019-12-28                       58080              73792   
       2019-12-29                       56229              71821   
       2019-12-30                       63462              81256   
       2019-12-31                      117732             179175   

                   Rock_Streams_Sum  Folk_Acoustic_Streams_Sum Sunny_Flag  \
region date                                                                 
AUT    2019-01-02              3329                          0          0   
       2019-01-03              6477                          0          0   
       2019-01-04              3588                          0          1   
       2019-01-05              7019                          0          0   
       2019-01-06              5700                          0          0   
...                             ...                        ...        ...   
SWE    2019-12-27             19286                      35663          0   
       2019-12-28             19451                      34514          0   
       2019-12-29             17549                      33100          0   
       2019-12-30             21111                      40055          0   
       2019-12-31            153686                      73845          0   

                  Rainy_Flag Cloudy_Flag Snowy_Flag Humid_Flag  
region date                                                     
AUT    2019-01-02          1           0          1          1  
       2019-01-03          1           1          1          1  
       2019-01-04          0           0          0          1  
       2019-01-05          0           0          0          1  
       2019-01-06          1           1          0          1  
...                      ...         ...        ...        ...  
SWE    2019-12-27          0           0          0          1  
       2019-12-28          0           1          0          1  
       2019-12-29          1           1          0          1  
       2019-12-30          1           1          0          1  
       2019-12-31          1           1          0          1  

[7277 rows x 11 columns]

In [32]:
df_final['Total_Streams_Sum'] = df_final['Pop_Streams_Sum'] + df_final['Dance_EDM_Streams_Sum'] + df_final['Hiphop_and_Rap_Streams_Sum'] + df_final['Latin_Streams_Sum'] + df_final['Rock_Streams_Sum'] + df_final['Folk_Acoustic_Streams_Sum']
df_final

Pop_Streams_Sum  Dance_EDM_Streams_Sum  \
region date                                                 
AUT    2019-01-02            61182                  35702   
       2019-01-03            60216                  35239   
       2019-01-04            61845                  36085   
       2019-01-05            58174                  35188   
       2019-01-06            51145                  29152   
...                            ...                    ...   
SWE    2019-12-27           133360                  35663   
       2019-12-28           129569                  34514   
       2019-12-29           123262                  33100   
       2019-12-30           143015                  40055   
       2019-12-31           652611                 232389   

                   Hiphop_and_Rap_Streams_Sum  Latin_Streams_Sum  \
region date                                                        
AUT    2019-01-02                       24773              10982   
       2019-01-03                       25646              11390   
       2019-01-04                       25997              11835   
       2019-01-05                       24777              11233   
       2019-01-06                       21185               9537   
...                                       ...                ...   
SWE    2019-12-27                       59410              74399   
       2019-12-28                       58080              73792   
       2019-12-29                       56229              71821   
       2019-12-30                       63462              81256   
       2019-12-31                      117732             179175   

                   Rock_Streams_Sum  Folk_Acoustic_Streams_Sum Sunny_Flag  \
region date                                                                 
AUT    2019-01-02              3329                          0          0   
       2019-01-03              6477                          0          0   
       2019-01-04              3588                          0          1   
       2019-01-05              7019                          0          0   
       2019-01-06              5700                          0          0   
...                             ...                        ...        ...   
SWE    2019-12-27             19286                      35663          0   
       2019-12-28             19451                      34514          0   
       2019-12-29             17549                      33100          0   
       2019-12-30             21111                      40055          0   
       2019-12-31            153686                      73845          0   

                  Rainy_Flag Cloudy_Flag Snowy_Flag Humid_Flag  \
region date                                                      
AUT    2019-01-02          1           0          1          1   
       2019-01-03          1           1          1          1   
       2019-01-04          0           0          0          1   
       2019-01-05          0           0          0          1   
       2019-01-06          1           1          0          1   
...                      ...         ...        ...        ...   
SWE    2019-12-27          0           0          0          1   
       2019-12-28          0           1          0          1   
       2019-12-29          1           1          0          1   
       2019-12-30          1           1          0          1   
       2019-12-31          1           1          0          1   

                   Total_Streams_Sum  
region date                           
AUT    2019-01-02             135968  
       2019-01-03             138968  
       2019-01-04             139350  
       2019-01-05             136391  
       2019-01-06             116719  
...                              ...  
SWE    2019-12-27             357781  
       2019-12-28             349920  
       2019-12-29             335061  
       2019-12-30             388954  
       2019

## Creating required columns to do t test in RStudio 

### Ideology behind t-test (example: Pop on a sunny day): We will try to check if "the difference between pop streams and non-pop streams" is significantly different on a sunny day.
### Similar t-tests will be done for: The top 6 genres on different weather conditions (sunny, rainy, cloudy, snowy, humid)

In [33]:
df_t_test = df_final

df_t_test['Pop_NonPop_Difference'] = df_t_test['Pop_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Pop_Streams_Sum'])
df_t_test['DanceEDM_NonDanceEDM_Difference'] = df_t_test['Dance_EDM_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Dance_EDM_Streams_Sum'])
df_t_test['HiphopRap_NonHiphopRap_Difference'] = df_t_test['Hiphop_and_Rap_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Hiphop_and_Rap_Streams_Sum'])
df_t_test['latin_Nonlatin_Difference'] = df_t_test['Latin_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Latin_Streams_Sum'])
df_t_test['Rock_NonRock_Difference'] = df_t_test['Rock_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Rock_Streams_Sum'])
df_t_test['FolkAcoustic_NonFolkAcoustic_Difference'] = df_t_test['Folk_Acoustic_Streams_Sum'] - (df_t_test['Total_Streams_Sum'] - df_t_test['Folk_Acoustic_Streams_Sum'])

df_t_test

Pop_Streams_Sum  Dance_EDM_Streams_Sum  \
region date                                                 
AUT    2019-01-02            61182                  35702   
       2019-01-03            60216                  35239   
       2019-01-04            61845                  36085   
       2019-01-05            58174                  35188   
       2019-01-06            51145                  29152   
...                            ...                    ...   
SWE    2019-12-27           133360                  35663   
       2019-12-28           129569                  34514   
       2019-12-29           123262                  33100   
       2019-12-30           143015                  40055   
       2019-12-31           652611                 232389   

                   Hiphop_and_Rap_Streams_Sum  Latin_Streams_Sum  \
region date                                                        
AUT    2019-01-02                       24773              10982   
       2019-01-03                       25646              11390   
       2019-01-04                       25997              11835   
       2019-01-05                       24777              11233   
       2019-01-06                       21185               9537   
...                                       ...                ...   
SWE    2019-12-27                       59410              74399   
       2019-12-28                       58080              73792   
       2019-12-29                       56229              71821   
       2019-12-30                       63462              81256   
       2019-12-31                      117732             179175   

                   Rock_Streams_Sum  Folk_Acoustic_Streams_Sum Sunny_Flag  \
region date                                                                 
AUT    2019-01-02              3329                          0          0   
       2019-01-03              6477                          0          0   
       2019-01-04              3588                          0          1   
       2019-01-05              7019                          0          0   
       2019-01-06              5700                          0          0   
...                             ...                        ...        ...   
SWE    2019-12-27             19286                      35663          0   
       2019-12-28             19451                      34514          0   
       2019-12-29             17549                      33100          0   
       2019-12-30             21111                      40055          0   
       2019-12-31            153686                      73845          0   

                  Rainy_Flag Cloudy_Flag Snowy_Flag Humid_Flag  \
region date                                                      
AUT    2019-01-02          1           0          1          1   
       2019-01-03          1           1          1          1   
       2019-01-04          0           0          0          1   
       2019-01-05          0           0          0          1   
       2019-01-06          1           1          0          1   
...                      ...         ...        ...        ...   
SWE    2019-12-27          0           0          0          1   
       2019-12-28          0           1          0          1   
       2019-12-29          1           1          0          1   
       2019-12-30          1           1          0          1   
       2019-12-31          1           1          0          1   

                   Total_Streams_Sum  Pop_NonPop_Difference  \
region date                                                   
AUT    2019-01-02             135968                 -13604   
       2019-01-03             138968                 -18536   
       2019-01-04             139350                 -15660   
       2019-01-05             136391                 -20043   
       2019-01-06             116719                 -14429   
...                              ...  

In [34]:
df_t_test.to_csv('df_t_test.csv')